In [4]:
%load_ext Cython

The Cython extension is already loaded. To reload it, use:
  %reload_ext Cython


In [2]:
%%cython
import numpy as np
cimport numpy as np
cimport cython

# @cython.boundscheck(False)
@cython.wraparound(False)
def flip_wrapped_1d(np.ndarray[np.int8_t, ndim=2] source,
                     np.ndarray[np.int_t, ndim=1] pad_size,
                     np.ndarray[np.int_t, ndim=2] pos):
    assert source.shape[0] == pos.shape[0], 'batch size of source and pos do not match'
    assert pad_size.shape[0] == 1,          'pad size should be of shape (N, 1)'
    assert pos.shape[1] == 1,               'pos should be of shape (N, 1)'
    cdef int wrap_x
    cdef int x
    cdef int n, N = source.shape[0]
    cdef int size_x = source.shape[1]-2*pad_size[0]
    for n in range(N):
        for wrap_x in range(-1, 2):
            x = pad_size[0]+wrap_x*size_x+pos[n, 0]
            if 0 <= x and x < source.shape[1]:
                source[n, x] *= -1

# @cython.boundscheck(False)
@cython.wraparound(False)
def flip_wrapped_2d(np.ndarray[np.int8_t, ndim=3] source,
                     np.ndarray[np.int_t, ndim=1] pad_size,
                     np.ndarray[np.int_t, ndim=2] pos):
    assert source.shape[0] == pos.shape[0], 'batch size of source and pos do not match'
    assert pad_size.shape[0] == 2,          'pad size should be of shape (N, 2)'
    assert pos.shape[1] == 2,               'pos should be of shape (N, 2)'
    cdef int wrap_x, wrap_y
    cdef int x, y
    cdef int n, N = source.shape[0]
    cdef int size_x = source.shape[2]-2*pad_size[1]
    cdef int size_y = source.shape[1]-2*pad_size[0]
    for n in range(N):
        for wrap_y in range(-1, 2):
            y = pad_size[0]+wrap_y*size_y+pos[n, 0]
            if 0 <= y and y < source.shape[1]:
                for wrap_x in range(-1, 2):
                    x = pad_size[1]+wrap_x*size_x+pos[n, 1]
                    if 0 <= x and x < source.shape[2]:
                        source[n, y, x] *= -1

# @cython.boundscheck(False)
@cython.wraparound(False)
def flip_wrapped_3d(np.ndarray[np.int8_t, ndim=4] source,
                     np.ndarray[np.int_t, ndim=1] pad_size,
                     np.ndarray[np.int_t, ndim=2] pos):
    assert source.shape[0] == pos.shape[0], 'batch size of source and pos do not match'
    assert pad_size.shape[0] == 3,          'pad size should be of shape (N, 3)'
    assert pos.shape[1] == 3,               'pos should be of shape (N, 3)'
    cdef int wrap_x, wrap_y, wrap_z
    cdef int x, y, z
    cdef int n, N = source.shape[0]
    cdef int size_x = source.shape[3]-2*pad_size[2]
    cdef int size_y = source.shape[2]-2*pad_size[1]
    cdef int size_z = source.shape[1]-2*pad_size[0]
    for n in range(N):
        for wrap_z in range(-1, 2):
            z = pad_size[0]+wrap_z*size_z+pos[n, 0]
            if 0 <= z and z < source.shape[1]:
                for wrap_y in range(-1, 2):
                    y = pad_size[1]+wrap_y*size_y+pos[n, 1]
                    if 0 <= y and y < source.shape[2]:
                        for wrap_x in range(-1, 2):
                            x = pad_size[2]+wrap_x*size_x+pos[n, 2]
                            if 0 <= x and x < source.shape[3]:
                                source[n, z, y, x] *= -1


x = np.arange(10).reshape(1, 10).astype(np.int8)+1
wrap = [(0,0), (2,2)]
y = np.pad(x, wrap, 'wrap')
flipped = y.copy()
flip_wrapped_1d(flipped, np.array([2]), np.array([[9]]))
flipped_2 = x.copy()
flipped_2[:, 9] *= -1
assert np.allclose(flipped, np.pad(flipped_2, wrap, 'wrap'))


x = np.arange(16).reshape(1, 4, 4).astype(np.int8)+1
wrap = [(0,0), (2,2), (3,3)]
y = np.pad(x, wrap, 'wrap')
flipped = y.copy()
flip_wrapped_2d(flipped, np.array([2, 3]), np.array([[0, 3]]))
flipped_2 = x.copy()
flipped_2[:, 0, 3] *= -1
assert np.allclose(flipped, np.pad(flipped_2, wrap, 'wrap'))

x = np.arange(64).reshape(1, 4, 4, 4).astype(np.int8)+1
wrap = [(0,0), (2,2), (3,3), (1,1)]
y = np.pad(x, wrap, 'wrap')
flipped = y.copy()
flip_wrapped_3d(flipped, np.array([2, 3, 1]), np.array([[0, 3, 0]]))
flipped_2 = x.copy()
flipped_2[:, 0, 3, 0] *= -1
assert np.allclose(flipped, np.pad(flipped_2, wrap, 'wrap'))

In [24]:
%%cython

import numpy as np
cimport numpy as np
cimport cython
ctypedef fused numeric:
    np.int8_t
    np.complex64_t


# @cython.boundscheck(False)
@cython.wraparound(False)
def replace_wrapped_1d(np.ndarray[numeric, ndim=2] old,
                        np.ndarray[numeric, ndim=2] replacement,
                        np.ndarray[np.int_t, ndim=2] center):
    assert old.shape[0] == replacement.shape[0] == center.shape[0], 'old, replacement and center should have same batch size'
    assert center.shape[1] == 1,          'center should be of shape (N,1)'
    assert replacement.shape[1] % 2 == 1, 'dimension 1 of replacement should be odd'
    cdef int wrap_x
    cdef int x
    cdef int n, N = old.shape[0]
    cdef int R_x = replacement.shape[1]
    cdef int pad_x = (R_x-1)//2
    cdef int size_x = old.shape[1]-2*pad_x
    cdef int offset_x
    for n in range(N):
        for wrap_x in range(-1, 2):
            offset_x = int(wrap_x*size_x+center[n, 0])
            for x in range(max(0, -offset_x), min(R_x, old.shape[1]-offset_x)):
                old[n, x+offset_x] = replacement[n, x]


# @cython.boundscheck(False)
@cython.wraparound(False)
def replace_wrapped_2d(np.ndarray[numeric, ndim=3] old,
                        np.ndarray[numeric, ndim=3] replacement,
                        np.ndarray[np.int_t, ndim=2] center):
    assert old.shape[0] == replacement.shape[0] == center.shape[0], 'old, replacement and center should have same batch size'
    assert center.shape[1] == 2,          'center should be of shape (N,2)'
    assert replacement.shape[1] % 2 == 1, 'dimension 1 of replacement should be odd'
    assert replacement.shape[2] % 2 == 1, 'dimension 2 of replacement should be odd'
    cdef int wrap_x, wrap_y
    cdef int x, y
    cdef int n, N = old.shape[0]
    cdef int R_x = replacement.shape[2]
    cdef int R_y = replacement.shape[1]
    cdef int pad_x = (R_x-1)//2
    cdef int pad_y = (R_y-1)//2
    cdef int size_x = old.shape[2]-2*pad_x
    cdef int size_y = old.shape[1]-2*pad_y
    cdef int offset_x, offset_y
    for n in range(N):
        for wrap_y in range(-1, 2):
            offset_y = int(wrap_y*size_y+center[n, 0])
            for y in range(max(0, -offset_y), min(R_y, old.shape[1]-offset_y)):
                for wrap_x in range(-1, 2):
                    offset_x = int(wrap_x*size_x+center[n, 1])
                    for x in range(max(0, -offset_x), min(R_x, old.shape[2]-offset_x)):
                        old[n, y+offset_y, x+offset_x] = replacement[n, y, x]
                        


# @cython.boundscheck(False)
@cython.wraparound(False)
def replace_wrapped_3d(np.ndarray[numeric, ndim=4] old,
                        np.ndarray[numeric, ndim=4] replacement,
                        np.ndarray[np.int_t, ndim=2] center):
    assert old.shape[0] == replacement.shape[0] == center.shape[0], 'old, replacement and center should have same batch size'
    assert center.shape[1] == 3,          'center should be of shape (N,3)'
    assert replacement.shape[1] % 2 == 1, 'dimension 1 of replacement should be odd'
    assert replacement.shape[2] % 2 == 1, 'dimension 2 of replacement should be odd'
    assert replacement.shape[3] % 2 == 1, 'dimension 3 of replacement should be odd'
    cdef int wrap_x, wrap_y, wrap_z
    cdef int x, y, z
    cdef int n, N = old.shape[0]
    cdef int R_x = replacement.shape[3]
    cdef int R_y = replacement.shape[2]
    cdef int R_z = replacement.shape[1]
    cdef int pad_x = (R_x-1)//2
    cdef int pad_y = (R_y-1)//2
    cdef int pad_z = (R_z-1)//2
    cdef int size_x = old.shape[3]-2*pad_x
    cdef int size_y = old.shape[2]-2*pad_y
    cdef int size_z = old.shape[1]-2*pad_z
    cdef int offset_x, offset_y, offset_z
    for n in range(N):
        for wrap_z in range(-1, 2):
            offset_z = int(wrap_z*size_z+center[n, 0])
            for z in range(max(0, -offset_z), min(R_z, old.shape[1]-offset_z)):
                for wrap_y in range(-1, 2):
                    offset_y = int(wrap_y*size_y+center[n, 1])
                    for y in range(max(0, -offset_y), min(R_y, old.shape[2]-offset_y)):
                        for wrap_x in range(-1, 2):
                            offset_x = int(wrap_x*size_x+center[n, 2])
                            for x in range(max(0, -offset_x), min(R_x, old.shape[3]-offset_x)):
                                old[n, z+offset_z, y+offset_y, x+offset_x] = replacement[n, z, y, x]
                                

wrap = [(0,0), (1,1)]
r = np.array([[-1, -2, -3]]).astype(np.complex64)
x = np.arange(4).reshape(1, 4).astype(np.complex64)
x_w = np.pad(x, wrap, 'wrap')
org = x_w.copy()
replace_wrapped_1d(x_w, r, np.array([[0]]))
x2 = np.roll(x, 1, 1)
x2[:, :3] = r
x2 = np.roll(x2, -1, 1)
x2_w = np.pad(x2, wrap, 'wrap')
assert np.allclose(x_w, x2_w), '1d'

wrap = [(0,0), (1,1), (2,2)]
r = np.arange(15).reshape(1, 3, 5).astype(np.complex64)+100
x = np.arange(25).reshape(1, 5, 5).astype(np.complex64)
x_w = np.pad(x, wrap, 'wrap')
org = x_w.copy()
replace_wrapped_2d(x_w, r, np.array([[0, 0]]))
x2 = np.roll(np.roll(x, 1, 1), 2, 2)
x2[:, :3, :5] = r
x2 = np.roll(np.roll(x2, -1, 1), -2, 2)
x2_w = np.pad(x2, wrap, 'wrap')
assert np.allclose(x_w, x2_w), '2d'

wrap = [(0,0), (1,1), (2,2), (3,3)]
r = np.arange(105).reshape(1, 3, 5, 7).astype(np.complex64)+100
x = np.arange(375).reshape(1, 5, 5, 15).astype(np.complex64)
x_w = np.pad(x, wrap, 'wrap')
org = x_w.copy()
replace_wrapped_3d(x_w, r, np.array([[0, 0, 0]]))
x2 = np.roll(np.roll(np.roll(x, 1, 1), 2, 2), 3, 3)
x2[:, :3, :5, :7] = r
x2 = np.roll(np.roll(np.roll(x2, -1, 1), -2, 2), -3, 3)
x2_w = np.pad(x2, wrap, 'wrap')
assert np.allclose(x_w, x2_w), '3d'

In [23]:
%%cython

import numpy as np
cimport numpy as np
cimport cython
ctypedef fused numeric:
    np.int8_t
    np.complex64_t


# @cython.boundscheck(False)
@cython.wraparound(False)
def replace_wrapped_1d(np.ndarray[numeric, ndim=2] old,
                        np.ndarray[numeric, ndim=2] replacement,
                        np.ndarray[np.int_t, ndim=2] center):
    assert old.shape[0] == replacement.shape[0] == center.shape[0], 'old, replacement and center should have same batch size'
    assert center.shape[1] == 1,          'center should be of shape (N,1)'
    assert replacement.shape[1] % 2 == 1, 'dimension 1 of replacement should be odd'
    cdef int wrap_x
    cdef int x
    cdef int n, N = old.shape[0]
    cdef int R_x = replacement.shape[1]
    cdef int pad_x = (R_x-1)//2
    cdef int size_x = old.shape[1]-2*pad_x
    cdef int offset_x
    for n in range(N):
        for wrap_x in range(-1, 2):
            offset_x = int(wrap_x*size_x+center[n, 0])
            for x in range(max(0, -offset_x), min(R_x, old.shape[1]-offset_x)):
                old[n, x+offset_x] = replacement[n, x]

wrap = [(0,0), (1,1)]
r = np.array([[-1, -2, -3]]).astype(np.int8)
x = np.arange(5).reshape(1, -1).astype(np.int8)
x_w = np.pad(x, wrap, 'wrap')
replaced = x_w.copy()
pos = np.array([[0]])


replace_wrapped_1d(replaced, r, pos)
print(x_w)
print(replaced)

[[4 0 1 2 3 4 0]]
[[-1 -2 -3  2  3 -1 -2]]


In [199]:
%%cython

import numpy as np
cimport numpy as np
cimport cython
ctypedef fused numeric:
    np.int8_t
    np.complex64_t


# @cython.boundscheck(False)
@cython.wraparound(False)
def window_1d(np.ndarray[numeric, ndim=2] source,
              np.ndarray[np.int_t, ndim=1] size,
              np.ndarray[np.int_t, ndim=2] center):
    assert source.shape[0] == center.shape[0], 'source and center should have same batch size'
    assert size.shape[0] == 1,                 'size should be of shape (N,1)'
    assert center.shape[1] == 1,               'center should be of shape (N,1)'
    assert size[0] % 2 == 1,                   'size[0] should be odd'
    
    cdef int n, N = source.shape[0]
    cdef int x
    cdef int offset_x
    
    cdef np.ndarray[numeric, ndim=2] out = np.zeros([N, size[0]], dtype=source.dtype)
    
    for n in range(N):
        offset_x = center[n, 0]
        for x in range(size[0]):
            out[n, x] = source[n, x+offset_x]
    return out


# @cython.boundscheck(False)
@cython.wraparound(False)
def window_2d(np.ndarray[numeric, ndim=3] source,
              np.ndarray[np.int_t, ndim=1] size,
              np.ndarray[np.int_t, ndim=2] center):
    assert source.shape[0] == center.shape[0], 'source and center should have same batch size'
    assert size.shape[0] == 2,                 'size should be of shape (N,2)'
    assert center.shape[1] == 2,               'center should be of shape (N,2)'
    assert size[0] % 2 == 1,                   'size[0] should be odd'
    assert size[1] % 2 == 1,                   'size[1] should be odd'
    
    cdef int n, N = source.shape[0]
    cdef int x, y
    cdef int offset_x, offset_y
    
    cdef np.ndarray[numeric, ndim=3] out = np.zeros([N, size[0], size[1]], dtype=source.dtype)
    
    for n in range(N):
        offset_y = center[n, 0]
        for y in range(size[0]):
            offset_x = center[n, 1]
            for x in range(size[1]):
                out[n, y, x] = source[n, y+offset_y, x+offset_x]
    return out


# @cython.boundscheck(False)
@cython.wraparound(False)
def window_3d(np.ndarray[numeric, ndim=4] source,
              np.ndarray[np.int_t, ndim=1] size,
              np.ndarray[np.int_t, ndim=2] center):
    assert source.shape[0] == center.shape[0], 'source and center should have same batch size'
    assert size.shape[0] == 3,                 'size should be of shape (N,2)'
    assert center.shape[1] == 3,               'center should be of shape (N,2)'
    assert size[0] % 2 == 1,                   'size[0] should be odd'
    assert size[1] % 2 == 1,                   'size[1] should be odd'
    assert size[2] % 2 == 1,                   'size[2] should be odd'
    
    cdef int n, N = source.shape[0]
    cdef int x, y, z
    cdef int offset_x, offset_y, offset_z
    
    cdef np.ndarray[numeric, ndim=4] out = np.zeros([N, size[0], size[1], size[2]], dtype=source.dtype)
    
    for n in range(N):
        offset_z = center[n, 0]
        for z in range(size[0]):
            offset_y = center[n, 1]
            for y in range(size[1]):
                offset_x = center[n, 2]
                for x in range(size[2]):
                    out[n, z, y, x] = source[n, z+offset_z, y+offset_y, x+offset_x]
    return out
            
x = np.arange(20).reshape((1, -1)).astype(np.int8)-2
size = np.array([5])
center = np.array([[12]])
w = window_1d(x, size, center)
assert np.allclose(w, x[:, 12:17]), '1d'

x = np.arange(100).reshape((1, 10, 10)).astype(np.complex64)
size = np.array([3, 5])
center = np.array([[4, 5]])
assert np.allclose(window_2d(x, size, center), x[:, 4:7, 5:10]), '2d'

x = np.arange(1000).reshape((1, 10, 10, 10)).astype(np.complex64)
size = np.array([3, 5, 3])
center = np.array([[3, 0, 5]])
assert np.allclose(window_3d(x, size, center), x[:, 3:6, 0:5, 5:8]), '3d'